In [5]:
try:
    import torch
    from torch import nn
    from torch.utils.data import DataLoader

    import torchvision
    from torchvision import transforms
    from torchvision import datasets

    import matplotlib.pyplot as plt
except:
    !pip install torch
    !pip install torchvision
    !pip install matplotlib
    
    import torch
    from torch import nn
    from torch.utils.data import DataLoader

    import torchvision
    from torchvision import transforms
    from torchvision import datasets

    import matplotlib.pyplot as plt

In [4]:
print(torch.__version__)

2.0.1
